In [1]:
# Install libraries
%pip install pandas matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

print("Libraries imported successfully!")

Libraries imported successfully!


In [15]:
# Load and Clean Data
file_name = "Spotify_Dataset_V3.csv"

print(f"Loading data from {file_name}...")
try:
    df = pd.read_csv(file_name, on_bad_lines='skip', sep=';')
    print("Data loaded successfully!")
except FileNotFoundError:
    print(f"File {file_name} not found.")
    raise

Loading data from Spotify_Dataset_V3.csv...
Data loaded successfully!


In [16]:
# DEBUG cell: Checking column names
for col in df.columns:
    print(f"'{col}'")

'Rank'
'Title'
'Artists'
'Date'
'Danceability'
'Energy'
'Loudness'
'Speechiness'
'Acousticness'
'Instrumentalness'
'Valence'
'# of Artist'
'Artist (Ind.)'
'# of Nationality'
'Nationality'
'Continent'
'Points (Total)'
'Points (Ind for each Artist/Nat)'
'id'
'Song URL'


In [17]:
# Checking column names
print("Column names in the dataset:")
print(df.columns)

Column names in the dataset:
Index(['Rank', 'Title', 'Artists', 'Date', 'Danceability', 'Energy',
       'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Valence', '# of Artist', 'Artist (Ind.)', '# of Nationality',
       'Nationality', 'Continent', 'Points (Total)',
       'Points (Ind for each Artist/Nat)', 'id', 'Song URL'],
      dtype='object')


In [18]:
# Cleanin Dates and Columns

try:
    original_date_col = 'Date' 
    date_format_str = '%d/%m/%Y'

    df[original_date_col] = pd.to_datetime(df[original_date_col], 
                                    format=date_format_str, 
                                    errors='coerce')
    
    # Drop rows where date conversion failed
    df.dropna(subset=[original_date_col], inplace=True)
    print("Date column cleaned successfully!")

    # Renaming columns for easier access
    df.rename(columns={
        'Date': 'date',
        'Artist (Ind.)': 'main_artist',
        'Title': 'title',
        'Points (Total)': 'points'
    }, inplace=True)

    # Final check
    print("Final column names after cleaning:")
    print(df[['date', 'main_artist', 'points', 'title']].info())

    print("Data sample after cleaning:")
    print(df[['date', 'main_artist', 'points', 'title']].head())

except KeyError as e:
    print(f"Column not found: {e}")
    raise
except Exception as e:
    print(f"An error occurred during data cleaning: {e}")
    raise

Date column cleaned successfully!
Final column names after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651936 entries, 0 to 651935
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         651936 non-null  datetime64[ns]
 1   main_artist  651936 non-null  object        
 2   points       651936 non-null  int64         
 3   title        651936 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 19.9+ MB
None
Data sample after cleaning:
        date     main_artist  points            title
0 2023-05-29  Eslabon Armado     200  Ella Baila Sola
1 2023-05-29      Peso Pluma     200  Ella Baila Sola
2 2023-05-29       Bad Bunny     199   WHERE SHE GOES
3 2023-05-29       Yng Lvcas     198  La Bebe - Remix
4 2023-05-29      Peso Pluma     198  La Bebe - Remix


In [21]:
# Analytics and Aggregations

print("Top 10 artists by total points:")
top_artists = df.groupby('main_artist')['points'].sum().sort_values(ascending=False).head(10)
print(top_artists)

# Calculating monthly audio feature trends
audio_features = [
    'Danceability', 
    'Energy', 
    'Loudness', 
    'Speechiness', 
    'Acousticness', 
    'Instrumentalness', 
    'Valence'
]

# Set 'date' as the index to group by month ('ME' = Month-End)
monthly_features = df.set_index('date')[audio_features].resample('ME').mean()
print(monthly_features.head())

# Calculating Weekly Popularity for Top 10 Artists
print("Weekly popularity for top 10 artists:")
top_10_artist_names = top_artists.index.tolist()

# Filter main DataFrame to only include those 10 artists
df_top10 = df[df['main_artist'].isin(top_10_artist_names)]

# Group by week ('W' = Week-Ending) and sum points
weekly_top10 = df_top10.groupby([
    'main_artist', 
    pd.Grouper(key='date', freq='W')
])['points'].sum().reset_index()
print(weekly_top10.head())

print("Analysis complete.")

Top 10 artists by total points:
main_artist
Bad Bunny        2019436
Post Malone      1295725
Ed Sheeran       1243482
J Balvin         1188150
The Weeknd       1135657
Drake            1063196
Billie Eilish     904437
Dua Lipa          846561
Ozuna             798520
Ariana Grande     731726
Name: points, dtype: int64
            Danceability    Energy     Loudness  Speechiness  Acousticness  \
date                                                                         
2017-01-31      0.676349  0.663153 -5421.390958     0.118872      0.189034   
2017-02-28      0.674168  0.664282 -5361.735093     0.120968      0.200604   
2017-03-31      0.684339  0.657160 -5327.187876     0.124113      0.198529   
2017-04-30      0.692557  0.667696 -5246.150064     0.120353      0.180898   
2017-05-31      0.692619  0.676048 -5148.806050     0.111387      0.177832   

            Instrumentalness   Valence  
date                                    
2017-01-31          0.007810  0.489443  
2017-02-2